refer to this for installation https://github.com/abetlen/llama-cpp-python#installation-with-openblas--cublas--clblast--metal 

In [3]:
%pip install llama_index
%pip install llama-cpp-python
%pip install transformers
%pip install torch
%pip install tqdm
%pip install pypdf


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import logging
import sys

from llama_index.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt

logging.basicConfig(stream=sys.stdout, level=logging.INFO)  # Change INFO to DEBUG if you want more extensive logging
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

llm = LlamaCPP(
    model_url="https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q5_K_M.gguf",
    
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    
    temperature=0.0,
    max_new_tokens=1024,
    
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,  # note, this sets n_ctx in the model_kwargs below, so you don't need to pass it there.
    
    # kwargs to pass to __call__()
    generate_kwargs={},
    
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 0}, # I need to play with this and see if it actually helps
    
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /Users/jiasenn/Library/Caches/llama_index/models/llama-2-13b-chat.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 13824,  5120,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_K     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_K     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q5_K     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weigh

In [5]:
# Create an index of your documents
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

storage_directory = "./storage"

documents = SimpleDirectoryReader('./testdata').load_data()

service_context = ServiceContext.from_defaults(llm=llm, chunk_size=1024,
                                               embed_model="local",
                                               callback_manager=callback_manager)


index = VectorStoreIndex.from_documents(documents, service_context=service_context)
# Persist the index to disk
index.storage_context.persist(persist_dir=storage_directory)

/Users/jiasenn/Downloads/GitHub/s34-cms-portal/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**********
Trace: index_construction
    |_node_parsing ->  0.335583 seconds
      |_chunking ->  0.000899 seconds
      |_chunking ->  0.002587 seconds
      |_chunking ->  0.002633 seconds
      |_chunking ->  0.001802 seconds
      |_chunking ->  0.001599 seconds
      |_chunking ->  0.000922 seconds
      |_chunking ->  0.001183 seconds
      |_chunking ->  0.001593 seconds
      |_chunking ->  0.001521 seconds
      |_chunking ->  0.000897 seconds
      |_chunking ->  0.000822 seconds
      |_chunking ->  0.002306 seconds
      |_chunking ->  0.001744 seconds
      |_chunking ->  0.001032 seconds
      |_chunking ->  0.000796 seconds
      |_chunking ->  0.001578 seconds
      |_chunking ->  0.001208 seconds
      |_chunking ->  0.001257 seconds
      |_chunking ->  0.001361 seconds
      |_chunking ->  0.001411 seconds
      |_chunking ->  0.000809 seconds
      |_chunking ->  0.001153 seconds
      |_chunking ->  0.001411 seconds
      |_chunking ->  0.001208 seconds
      |_chu

In [6]:
# Now you can load the index from disk when needed, and not rebuild it each time.
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index import StorageContext, load_index_from_storage

# transcript_directory = "transcripts/ancient-aliens-official"
storage_directory = "./storage"

service_context = ServiceContext.from_defaults(llm=llm, chunk_size=1024,
                                               embed_model="local",
                                               callback_manager=callback_manager)

storage_context = StorageContext.from_defaults(persist_dir=storage_directory)
index = load_index_from_storage(storage_context, service_context=service_context)

INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
**********
Trace: index_construction
**********


In [7]:
# Query your index!
from IPython.display import Markdown, display
from llama_index.prompts import PromptTemplate

query_engine = index.as_query_engine(service_context=service_context,
                                     similarity_top_k=3)

# response = query_engine.query("What do you think of Facebook's LLaMa?")
# print(response)

# query = "What is the Alien nuclear agenda? \n\n" \
#         "Please summarize the information into 3 detailed paragraphs, something suitable for a blog post."

# query = "could you combine the learning styles from the document and generate a few game types based on the octalysis framework for different types of games"

query = "Generate 5 question according to 1 type of kolb_learning_style, regarding PDPA content."

response = query_engine.query(query)
display(Markdown(f"<b>{response}</b>"))

In [ ]:
raise Exception("Stop here")

In [ ]:
# Query your index!
from IPython.display import Markdown, display
from llama_index.prompts import PromptTemplate

query_engine = index.as_query_engine(service_context=service_context,
                                     similarity_top_k=3)

# response = query_engine.query("What do you think of Facebook's LLaMa?")
# print(response)

# query = "What is the Alien nuclear agenda? \n\n" \
#         "Please summarize the information into 3 detailed paragraphs, something suitable for a blog post."

query = "I am trying to develop a game to enhance adult learning learning \n\n" \
        "Given that the game is a narrative driven adventure game, please generate 5 multiple choice question regarding sexual harrassment \n\n"

response = query_engine.query(query)
display(Markdown(f"<b>{response}</b>"))